In [1]:
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import structured_light_tomography.models as models
import structured_light_tomography.training as training
import structured_light_tomography.dataset_generation as dg
import structured_light_tomography.photocount_treatment as pt
from torchvision.transforms import v2
import numpy as np
import matplotlib.pyplot as plts
from os.path import join
import torchvision
import torch.nn.functional as F
import h5py
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from structured_light_tomography.training import fidelity

device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def format_data(x):
    mean = [x[:, n, :, :].mean() for n in range(x.shape[1])]
    std = [x[:, n, :, :].std() for n in range(x.shape[1])]

    X = v2.Compose([
            torch.from_numpy,
            v2.Normalize(mean=mean, std=std),
            v2.Resize((64, 64)),
        ])(x).to(device)
    return X

Using cuda device


In [3]:
orders = range(1,6)

In [4]:
for order in orders:
    with h5py.File('TrainingData/intense_mixed.h5', 'r') as f:
            x = f[f'x_order{order}'][:]
            y = f[f'y_order{order}'][:]
            
    X = format_data(x)
    Y = torch.from_numpy(y).to(device)

    dset = TensorDataset(X, Y)

    train_size = int(0.85 * len(dset))
    test_size = len(dset) - train_size

    train_dataset, test_dataset = random_split(dset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256)

    L = x.shape[2]
    loss_fn = torch.nn.MSELoss()
    n_channels = 2
    n_classes = (order+1)**2-1

    model = models.ConvNet(L,L,n_channels, n_classes,[24,40,35],5,nn.ELU,[120,80,40]).to(device)

    optimizer = torch.optim.Adam(model.parameters(), amsgrad=True)

    save_path = f"runs/Mixed/Order{order}"

    writer = SummaryWriter(save_path)
    early_stopping = training.EarlyStopping(patience=30,save_path=save_path)
    for t in range(200):
        epoch = t+1
        print(f"-------------------------------\nEpoch {epoch}")
        training.train(model, train_loader, loss_fn, optimizer, device)
        val_loss = training.test(model, test_loader, loss_fn, device, epoch, writer, verbose=True)
        early_stopping(val_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
    print("Done!")
    writer.close()
        

/home/marcsgil/Desktop/structured_light_tomography/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


-------------------------------
Epoch 1
Training loss: 0.00385541
-------------------------------
Epoch 2
Training loss: 0.00313362
-------------------------------
Epoch 3
Training loss: 0.00290820
-------------------------------
Epoch 4
Training loss: 0.00272845
-------------------------------
Epoch 5
Training loss: 0.00238686
-------------------------------
Epoch 6
Training loss: 0.00198748
-------------------------------
Epoch 7
Training loss: 0.00221817
-------------------------------
Epoch 8
Training loss: 0.00242005
-------------------------------
Epoch 9
Training loss: 0.00212049
-------------------------------
Epoch 10
Training loss: 0.00178012
-------------------------------
Epoch 11
Training loss: 0.00199584
-------------------------------
Epoch 12
Training loss: 0.00209538
-------------------------------
Epoch 13
Training loss: 0.00285449
-------------------------------
Epoch 14
Training loss: 0.00168778
-------------------------------
Epoch 15
Training loss: 0.00168832
----

In [29]:
def A(dim, j):
    assert dim > j, "The dimension must be greater than the index."
    D = np.zeros(dim, dtype=np.float32)
    for i in range(j):
        D[i] = 1
    D[j] = -j
    D = D / np.linalg.norm(D)
    return np.diag(D)

def B(dim, j, k):
    B = np.zeros((dim, dim), dtype=np.float32)
    B[j-1, k-1] = 1
    B[k-1, j-1] = 1
    B = (B + B.T) / 2
    B = B / np.linalg.norm(B)
    return B

def C(dim, j, k):
    C = np.zeros((dim, dim), dtype=np.complex64)
    C[j-1, k-1] = 1j
    C[k-1, j-1] = -1j
    C = (C + C.T.conj()) / 2
    C = C / np.linalg.norm(C)
    return C

def get_basis(dim):
    As = np.stack([A(dim, j) for j in range(1, dim)])
    Bs = np.empty((dim * (dim - 1) // 2, dim, dim), dtype=np.float32)
    Cs = np.empty((dim * (dim - 1) // 2, dim, dim), dtype=np.complex64)
    for j in range(1, dim + 1):
        for k in range(0, j):
            Bs[(j - 2) * (j - 1) // 2 + k - 1, :, :] = B(dim, j, k)
            Cs[(j - 2) * (j - 1) // 2 + k - 1, :, :] = C(dim, j, k)
    return np.concatenate((As, Bs, Cs), axis=0)

def real_representation(rhos):
    assert rhos.shape[1] == rhos.shape[2], "The density matrices must be square."
    dim = rhos.shape[1]
    xs = np.empty((rhos.shape[0],dim**2 - 1), dtype=np.float32)
    basis = get_basis(dim)
    for rho, x in zip(rhos,xs):
        for i, b in enumerate(basis):
            x[i] = np.real(np.trace(rho @ b))
    return xs

def complex_representation(xs):
    device = xs.device
    dim = int((xs.size(1) + 1) ** 0.5)
    basis = torch.from_numpy(get_basis(dim)).to(device)

    return torch.stack([torch.eye(dim).to(device) / dim + sum(c * b for c, b in zip(x, basis)) for x in xs])

def fidelity(rhos1, rhos2):
    fids = torch.empty(rhos1.shape[0], dtype=torch.float32)
    for i, (rho1, rho2) in enumerate(zip(rhos1, rhos2)):
        sqrt_rho1 = torch.sqrt(rho1)
        fids[i] = torch.real(torch.trace(torch.sqrt(sqrt_rho1 @ rho2 @ sqrt_rho1)))
    return fids

In [37]:
rhos = torch.ones(100, 4, 4).to(device)
fidelity(rhos,rhos).shape

torch.Size([100])

In [38]:
orders = range(1,6)

metric = fidelity
mses = torch.empty(len(orders))


for order in orders:
    with h5py.File('ExperimentalData/Intense/mixed.h5', 'r') as f:
        x_exp = f[f'images_order{order}'][:]
        y_exp = f[f'labels_order{order}'][:]
    
    X_exp = format_data(np.float32(x_exp))
    Y_exp = torch.from_numpy(y_exp).to(device)
    model = torch.load(f"runs/Mixed/Order{order}/checkpoint.pt")

    with torch.no_grad():
        Y_hat = complex_representation(model(X_exp))
        print(Y_exp.shape)
        print(Y_hat.shape)
        mses[order-1] = metric(Y_hat,Y_exp).mean()
        

mses = mses.numpy()

/home/marcsgil/Desktop/structured_light_tomography/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/tmp/ipykernel_42230/1144312176.py:23: RuntimeWarning: invalid value encountered in divide
  C = C / np.linalg.norm(C)


torch.Size([100, 2, 2])
torch.Size([100, 2, 2])
torch.Size([100, 3, 3])
torch.Size([100, 3, 3])
torch.Size([100, 4, 4])
torch.Size([100, 4, 4])
torch.Size([100, 5, 5])
torch.Size([100, 5, 5])
torch.Size([100, 6, 6])
torch.Size([100, 6, 6])


In [39]:
mses

array([1.2491341,       nan,       nan,       nan,       nan],
      dtype=float32)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import numpy as np

# Enable LaTeX fonts
rc('text', usetex=True)

# Increase font size
rcParams['font.size'] = 12

# Apply log2 transformation to photocounts
log_photocounts = np.log2(photocounts)

# Set the x and y labels
plt.xlabel("Photocounts")
plt.ylabel("Fidelity")

# Create a list of markers
markers = ['o', '^', 'D', 's']

# Assuming fids is your matrix
for i, line in enumerate(fids, start=1):
    # Use the order index to select a marker from the list
    plt.plot(log_photocounts, line, label=f"Order {i}", marker=markers[i-1])

# Set the xticks to be the original photocounts values
plt.xticks(log_photocounts, photocounts)

# Set the y-axis limits
plt.ylim([0.9, 1])

# Set the y-axis ticks
plt.yticks(np.arange(0.88, 1.01, 0.01))

# Display the labels outside the plotting area
plt.legend(bbox_to_anchor=(1.0, 0.5), loc='center left')

# Display both major and minor grids
plt.grid(which='both')

# Set the minor grid lines to a dotted style and decrease their alpha value
plt.grid(which='minor', linestyle=':', alpha=0.5)

# Save the plot as a PDF file with all elements included
plt.savefig("plots/fidelities_ml.png", bbox_inches='tight')

plt.show()